# Calcio femminile 

Esempio di scraping dal sito [femminile.football.it](femminile.football.it)

In [13]:
from selenium import webdriver                         # import the webdriver API 
from selenium.webdriver.common.keys import Keys        # import the keys of the keyboard such as RETURNS, F1, ALT, ...
import re                                              # import regular expression

driver = webdriver.Chrome()                            # run the browser
                                           

### Funzione getPlayers
Fornita una lettera accede alla pagina di [ricerca delle giocatrice](https://femminile.football.it/ricerca.php) e recupera tutte le informazioni delle giocatrici con quella lettera, in particolare per ogni giocatrice recupera il suo id con cui si accede alla scheda relativa.

In [18]:
def getPlayers(letter):
    # parameters and output data
    url         = 'https://femminile.football.it/ricerca.php?Ricerca=giocatori&iniziale='
    xpath       = "/html/body/table/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[2]/table[2]"
    patternName = re.compile("[A-Z]+\s[A-Z]+") # QAFOKU MARTINA
    patternData = re.compile("([0-9]){2}\/([0-9]){2}\/([0-9]){4}") # 28/11/2000
    playersData = []
    
    driver.get(url+letter)  # get the web page

    tabella = driver.find_element_by_xpath(xpath)     # seleziono l'elemento tabella contenente le giocatrici
    players = tabella.find_elements_by_tag_name('a')  # seleziono tutti i tag 'a' (anchor) a partire dalla tabella

    playersData = [] # definisco una lista vuota dove andrò ad inserire i dati delle calciatrici    
    for player in players:
        n = patternName.search(player.text) # ricerco il nome
        d = patternData.search(player.text) # ricerco la data
        if(d):
            dataNascita = d.group()
        else:
            dataNascita = ''
        if(n):
            nome = n.group()
        else:
            nome = ''
        link = player.get_attribute('href')
        playersData.append({'nome':nome,'id':link.split('=')[1], 'dataNascita':dataNascita })
    
    return playersData

In [19]:
# test getPlayers
playersData = getPlayers('Q')
print(playersData)

18
[{'nome': 'QAFOKU MARTINA', 'id': '8316', 'dataNascita': '28/11/2000'}, {'nome': 'QUADRELLI SARA', 'id': '7267', 'dataNascita': '28/08/1997'}, {'nome': 'QUAGGIOTTO ELENA', 'id': '2262', 'dataNascita': '30/04/1992'}, {'nome': 'QUAGLIA FRANCESCA', 'id': '5688', 'dataNascita': '13/04/1996'}, {'nome': 'QUAGLIOTTO JESSICA', 'id': '3031', 'dataNascita': '16/03/1993'}, {'nome': 'QUARANTA MARTINA', 'id': '6611', 'dataNascita': '19/05/1995'}, {'nome': 'QUARESIMA MARA', 'id': '10475', 'dataNascita': '15/04/2002'}, {'nome': 'QUARTICELLI VALENTINA', 'id': '2398', 'dataNascita': '20/02/1990'}, {'nome': 'QUARTO FEDERICA', 'id': '3220', 'dataNascita': '13/04/1992'}, {'nome': 'QUARTULLO FEDERICA', 'id': '7046', 'dataNascita': '20/06/1991'}, {'nome': 'QUATTROCCHI FRANCESCA', 'id': '3829', 'dataNascita': '03/08/1988'}, {'nome': 'QUAZZICO FRANCESCA', 'id': '8752', 'dataNascita': '07/05/2001'}, {'nome': 'QUERCIA MARZIA', 'id': '4650', 'dataNascita': '16/09/1980'}, {'nome': 'QUERZOLA LAURA', 'id': '9149

### Funzione getPlayerData
fornito un id accede alla pagina della giocatrice e recupera i suoi dati

In [24]:
def getPlayerData(id):
    url   = "https://femminile.football.it/schedagiocatore.php?id_giocatore="
    xpath = "/html/body/table/tbody/tr/td/table/tbody/tr/td[2]/table[3]/tbody/tr[1]/td[2]/table"

    driver.get(url+id)
    
    tabella = driver.find_element_by_xpath(xpath)     # seleziono l'elemento tabella contenente le giocatrici
    rows    = tabella.find_elements_by_tag_name('tr') # seleziono tutti i tag 'tr' (table row) a partire dalla tabella
    
    playerData = {}
    for row in rows:
        if(row.text.strip()!=''):
            # print("-"+row.text)
            cells = row.find_elements_by_tag_name("td")
            playerData[cells[0].text]=cells[1].text.strip()    

    # Recupero la squadra in cui gioca nella stagione attuale dalla penultima riga
    xpath = "/html/body/table/tbody/tr/td/table/tbody/tr/td[2]/table[5]/tbody/tr[1]/td[2]/table//tr"
    rows  = driver.find_elements_by_xpath(xpath)
    row   = rows[len(rows)-2]
    cells = row.find_elements_by_tag_name("td")
    playerData["team2019/20"]=cells[0].text
    return playerData

In [25]:
#test getPlayerData
playerData = getPlayerData('9205')
print(playerData)

{'Data di nascita': '14/09/1999', 'luogo di nascita': '( [N.D.] ) - Italia', 'nazione': 'Italia', 'altezza': '', 'peso forma': '', 'ruolo': 'Attaccante', 'sito ufficiale': '', 'Nazionalita': 'Italiana', 'team2019/20': 'Saluzzo'}


### Programma principale

In [16]:
#letters = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
letters = ['Q','Z']

# recupero tutte le giocatrici
players = [] # definisco una lista vuota dove inserirò tutte le giocatrici
for letter in letters:
    players.extend(getPlayers(letter))         # add a list to a list

# per ogni giocatrice recupero i dati
playersDB = [] # definisco una lista vuota dove andrò a salvare i dati di ogni giocatrice
for player in players:
    player.update(getPlayerData(player['id'])) # add a dict to a dict
    playersDB.append(player)
    

## Visualizzo i risultati

In [23]:
print(len(playersDB))

188


## Programma principale in modalità headless

In [22]:
driver.quit() # chiudo il driver che non è in modalità headless

from selenium.webdriver.chrome.options import Options
chromeOptions = Options()  
chromeOptions.add_argument("--headless") 

driver = webdriver.Chrome(options=chromeOptions) # lancio il driver in modalità Headless

# recupero tutte le giocatrici
players = [] # definisco una lista vuota dove inserirò tutte le giocatrici
for letter in letters:
    players.extend(getPlayers(letter))         # add a list to a list


# per ogni giocatrice recupero i dati
playersDB = [] # definisco una lista vuota dove andrò a salvare i dati di ogni giocatrice
for player in players:
    player.update(getPlayerData(player['id'])) # add a dict to a dict
    playersDB.append(player)

## Salvo su file i dati in formato CSV

In [ ]:
import csv
keys = playersDB[0].keys() # Recupero i nomi dei campi dal primo dictionary
with open('playersDB.csv', 'w', newline='') as csvFile:
    dict_writer = csv.DictWriter(csvFile, keys) # imposto i nomi dei campi
    dict_writer.writeheader()                   # scriva la prima riga
    dict_writer.writerows(playersDB)            # scrivo i dati